### CONFIGURATION

In [119]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 200)
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [120]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn

In [125]:
#VCF
VCF_PATH = "/Users/peterpriestley/hmf/analyses/ensembleRuleTesting/"

# BED
BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/"
BED_FILE_NAME = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.bed"
#BED_PATH = "/Users/peterpriestley/hmf/analyses/slices/"
#BED_FILE_NAME = "cosmicCancerCensus.bed" 
#BED_FILE_NAME = "CPCT_Slicing.bed"
USE_BED = True

# HEALTH CHECKS
HEALTH_CSV_PATH="/Users/peterpriestley/hmf/analyses/healthCheckAnalysis/"
HEALTH_CSV_FILENAME="healthCheckFile.csv"

In [126]:
# SPECIFY SAMPLES
VCF_SAMPLES=[]
for x in os.listdir(VCF_PATH):
    if x[-10:] == "melted.vcf":
        VCF_SAMPLES.append(x)
#print VCF_SAMPLES

VCF_SAMPLES = ['CPCT02070025R_CPCT02070025T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf']

<h3> LOAD VCF

In [127]:
#Load all VCF_SAMPLES to DF
df = pd.DataFrame({})
for VCF_SAMPLE in VCF_SAMPLES:
    VCF_FILE_NAME = VCF_SAMPLE
    SAMPLE_NAMES = {VCF_SAMPLE[14:len(VCF_SAMPLE)-50]:'melted'}
    if USE_BED:
        bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
        df = pd.concat([df,aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[:12],True,True,bed,True)])
    else:
        df = pd.concat([df,aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[:12],True)])

reading BED file
Bed File Loaded
reading vcf file: CPCT02070025R_CPCT02070025T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
Number variants loaded: 41811


In [124]:
#LOAD HEALTH_CHECKS
hc = pd.read_csv(HEALTH_CSV_PATH + HEALTH_CSV_FILENAME)
hc = hc[['SAMPLE','TUMOR_PERCENTAGE','TUMOR_TYPE','PIPELINE_VERSION']]
hc = hc[hc['SAMPLE'].str[12]=="T"]   # only tumor rows
hc['SAMPLE']=hc['SAMPLE'].str[:12]
hc = hc.set_index(['SAMPLE'])
df = pd.merge(df,hc,how='left', left_on='patientName',right_index=True)
df.fillna("Unknown", inplace=True)

KeyError: 'patientName'

In [128]:
dfPatients = pd.DataFrame([x[:12] for x in VCF_SAMPLES])
dfPatients.columns = (['patientName'])
dfPatients = pd.merge(dfPatients,hc,how='left', left_on='patientName',right_index=True)
dfPatients.fillna("Unknown", inplace=True)
dfPatients = pd.DataFrame(dfPatients.groupby(['TUMOR_TYPE']).agg('count')['patientName'])
dfPatients.rename(columns={'patientName':'patientCount'},inplace = True)

<h3> High Level Stats

In [129]:
dfPatients

,patientCount
TUMOR_TYPE,
Unknown,1


In [130]:
dfVariantCountByTumor = df.groupby(['TUMOR_TYPE','annGene']).agg('count')['patientName'].reset_index()
dfVariantCountByTumor = pd.merge(dfVariantCountByTumor,dfPatients,how='left', left_on='TUMOR_TYPE',right_index=True)
dfVariantCountByTumor['freq']=dfVariantCountByTumor['patientName']/dfVariantCountByTumor['patientCount']
pd.pivot_table(dfVariantCountByTumor, values=['freq','patientCount'], index=['annGene'], columns=['TUMOR_TYPE'], aggfunc='sum').fillna("")

KeyError: 'TUMOR_TYPE'

In [137]:
df.head(1)

,chrom,pos,chromPos,chromFrac,id,ref,vennSegment,numCallers,variantType,variantSubType,filter,bedRegion,inDBSNP,inCOSMIC,annGene,annWorstImpact,annWorstEffect,annAllEffects,consensus,meltedallele,meltedAF,meltedDP,meltedQS,meltedSGT,meltedindelDiff,meltedindelPos,patientName
0,1,10583,1:10583,1.000042,rs58108140,G,mutect,1,SNP,,PASS,,True,False,,,,,False,A,0.105263,76,-1.0,0/1,,-1,CPCT02070025


In [154]:
pd.pivot_table(df[((df['annWorstImpact']=='MODERATE')|(df['annWorstImpact']=='HIGH'))&(df.consensus==True)], values='pos', index=['annWorstImpact','annWorstEffect','bedRegion'], columns=['consensus'], aggfunc='count').fillna("")

consensus                                                        True
annWorstImpact annWorstEffect                         bedRegion      
HIGH           frameshift_variant                     Default       1
               splice_donor_variant&intron_variant    Default       1
               stop_gained                                          1
MODERATE       missense_variant                                     3
                                                      Default      19
               missense_variant&splice_region_variant Default       1
               sequence_feature                                     1

In [41]:
pd.pivot_table((df[(df.patientName=='CPCT02020360')],values='pos', index=['patientName'], columns=['consensus','vennSegment'], aggfunc='count',margins=True).fillna("")

SyntaxError: invalid syntax (<ipython-input-41-174ddef69118>, line 1)

In [118]:
df[(df.patientName=='CPCT02010396')][['patientName','TUMOR_TYPE','pos','chrom','id','variantType','ref','meltedallele','meltedDP','meltedAF','vennSegment','annGene']].head(20)

,patientName,TUMOR_TYPE,pos,chrom,id,variantType,ref,meltedallele,meltedDP,meltedAF,vennSegment,annGene
0,CPCT02010396,Colon and Rectal cancer,112175358,5,COSM18779,SNP,C,G,127,0.086614,strelka-mutect,APC
1,CPCT02010396,Colon and Rectal cancer,25398284,12,rs121913529;COSM522,SNP,C,G,121,0.057851,strelka-mutect,KRAS
2,CPCT02010396,Colon and Rectal cancer,32907262,13,.,SNP,G,A,165,0.036585,strelka-mutect,BRCA2
3,CPCT02010396,Colon and Rectal cancer,7578274,17,.,INDEL,TGAGGAGG,TGAGG,112,0.110092,strelka-varscan-freebayes,TP53


In [ ]:
df[['variantType','pos']].groupby(['variantType']).agg('count')

<h3> Allelic Depth

In [ ]:
#Alllele Freq By Caller
for columnName in list(snpdf):
    if columnName.endswith('allele'):
        ser = snpdf[snpdf[columnName] != ''][columnName[:-6] + 'AF']
        ser = ser.sort_values()
        ser[len(ser)] = ser.iloc[-1]
        cum_dist = np.linspace(0.,1.,len(ser))
        ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-6]+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
        ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller",figsize=[15,6])